# **CITY BIKE API**

### **ABOUT CITY BIKE API**

The city bike API is a  project that provides bike sharing data for apps, research and projects to use. To date Citybikes supports more than 400 cities and the Citybikes API is the most widely used dataset for building bike sharing transportation projects.

In [1]:
import requests
import json
import pandas as pd

In [2]:
# API url
url = "http://api.citybik.es/v2/networks"

In [3]:
# parsed the data as JSON
response = requests.get(url).json()
# prints first 5 values
response['networks'][:5]

[{'company': ['ЗАО «СитиБайк»'],
  'href': '/v2/networks/velobike-moscow',
  'id': 'velobike-moscow',
  'location': {'city': 'Moscow',
   'country': 'RU',
   'latitude': 55.75,
   'longitude': 37.616667},
  'name': 'Velobike'},
 {'company': ['Comunicare S.r.l.'],
  'href': '/v2/networks/bicincitta-siena',
  'id': 'bicincitta-siena',
  'location': {'city': 'Siena',
   'country': 'IT',
   'latitude': 43.3186,
   'longitude': 11.3306},
  'name': 'Bicincittà',
  'source': 'https://www.bicincitta.com/frmLeStazioni.aspx?ID=202'},
 {'company': ['Cyclopolis Systems'],
  'href': '/v2/networks/cyclopolis-maroussi',
  'id': 'cyclopolis-maroussi',
  'location': {'city': 'Maroussi',
   'country': 'GR',
   'latitude': 38.0568722388,
   'longitude': 23.8083299536},
  'name': 'Cyclopolis'},
 {'company': ['Cyclopolis Systems'],
  'href': '/v2/networks/cyclopolis-nafplio',
  'id': 'cyclopolis-nafplio',
  'location': {'city': 'Nafplio',
   'country': 'GR',
   'latitude': 37.5639397319,
   'longitude': 22

> From the documentation, the API provides field filtering parameter and from the list of features in the JSON formatted data, I am interested in the name of the company providing the bike and the location of the company.

In [4]:
# API url with the filter parameter
urlFilter = "http://api.citybik.es/v2/networks?fields=name,location"

In [5]:
# parsed the data as JSON
responseFilter = requests.get(urlFilter).json()
# prints first 5 values
responseFilter['networks'][:5]

[{'location': {'city': 'Moscow',
   'country': 'RU',
   'latitude': 55.75,
   'longitude': 37.616667},
  'name': 'Velobike'},
 {'location': {'city': 'Siena',
   'country': 'IT',
   'latitude': 43.3186,
   'longitude': 11.3306},
  'name': 'Bicincittà'},
 {'location': {'city': 'Maroussi',
   'country': 'GR',
   'latitude': 38.0568722388,
   'longitude': 23.8083299536},
  'name': 'Cyclopolis'},
 {'location': {'city': 'Nafplio',
   'country': 'GR',
   'latitude': 37.5639397319,
   'longitude': 22.8093402872},
  'name': 'Cyclopolis'},
 {'location': {'city': 'Parco dei Colli di Bergamo',
   'country': 'IT',
   'latitude': 45.72295637032245,
   'longitude': 9.649230073016383},
  'name': 'Bicincittà'}]

> The result above shows that location information is stored in a dictionary.

## **PUT INFORMATION INTO A TABLE**

In [6]:
# create main dataframe
bikeData = pd.DataFrame()

# loop through the JSON formatted data
for i in range(len(responseFilter['networks'])):
    # create a temporary dataframe
    temp = pd.DataFrame(
        {
            'Name': [responseFilter['networks'][i]['name']],
            'Country': [responseFilter['networks'][i]['location']['country']],
            'City': [responseFilter['networks'][i]['location']['city']],
            'Latitude': [responseFilter['networks'][i]['location']['latitude']],
            'Longitude': [responseFilter['networks'][i]['location']['longitude']]
        }
    )

    # concatenate the temporary dataframe with the earlier created dataframe
    bikeData = pd.concat([bikeData, temp], ignore_index=True)

bikeData.head()

,Name,Country,City,Latitude,Longitude
0,Velobike,RU,Moscow,55.750000,37.616667
1,Bicincittà,IT,Siena,43.318600,11.330600
2,Cyclopolis,GR,Maroussi,38.056872,23.808330
3,Cyclopolis,GR,Nafplio,37.563940,22.809340
4,Bicincittà,IT,Parco dei Colli di Bergamo,45.722956,9.649230


## **EXPLORATORY DATA ANALYSIS**

In [7]:
bikeData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       770 non-null    object 
 1   Country    770 non-null    object 
 2   City       770 non-null    object 
 3   Latitude   770 non-null    float64
 4   Longitude  770 non-null    float64
dtypes: float64(2), object(3)
memory usage: 30.2+ KB


> The result shows there are 770 rows and 5 features, no missing values, also the datatype of each feature appears to be appropriate.

#### **HOW MANY UNIQUE COUNTRIES AND CITIES ARE IN THE DATAFRAME?**

In [8]:
bikeData['Country'].nunique()

57

In [9]:
bikeData['City'].nunique()

750

> The outputs from the above cells tells that some countries and cities occurred more than once in the DataFrame, it would be a great idea to check for duplicates in the dataset so it doesn't affect the outcome of the analysis.

#### **CHECK FOR DUPLICATES IN THE DATASET**

In [10]:
bikeData.duplicated().sum()

0

> Even though the dataset have some countries and cities repeated, the result shows that there are no duplicate datapoint in the dataset.

In [12]:
bikeData.query('Country == "US"').head(7)

,Name,Country,City,Latitude,Longitude
27,WE-cycle,US,"Aspen, CO",39.194951,-106.837002
77,ATX MetroBike,US,"Austin, TX",30.264080,-97.743550
78,Bike Chattanooga,US,"Chattanooga, TN",35.045630,-85.309680
81,BIKETOWN,US,"Portland, OR",45.521754,-122.681079
83,Boulder BCycle,US,"Boulder, CO",40.008110,-105.263850
85,Broward BCycle,US,"Fort Lauderdale, FL",26.120260,-80.148190
86,Bublr Bikes,US,"Milwaukee, WI",43.036900,-87.896670


# **LOOKING INTO A PARTICULAR COMPANY**

In [15]:
response['networks'][:3]

[{'company': ['ЗАО «СитиБайк»'],
  'href': '/v2/networks/velobike-moscow',
  'id': 'velobike-moscow',
  'location': {'city': 'Moscow',
   'country': 'RU',
   'latitude': 55.75,
   'longitude': 37.616667},
  'name': 'Velobike'},
 {'company': ['Comunicare S.r.l.'],
  'href': '/v2/networks/bicincitta-siena',
  'id': 'bicincitta-siena',
  'location': {'city': 'Siena',
   'country': 'IT',
   'latitude': 43.3186,
   'longitude': 11.3306},
  'name': 'Bicincittà',
  'source': 'https://www.bicincitta.com/frmLeStazioni.aspx?ID=202'},
 {'company': ['Cyclopolis Systems'],
  'href': '/v2/networks/cyclopolis-maroussi',
  'id': 'cyclopolis-maroussi',
  'location': {'city': 'Maroussi',
   'country': 'GR',
   'latitude': 38.0568722388,
   'longitude': 23.8083299536},
  'name': 'Cyclopolis'}]

In [16]:
response['networks'][86]

{'company': ['BCycle, LLC'],
 'gbfs_href': 'https://gbfs.bcycle.com/bcycle_bublr/gbfs.json',
 'href': '/v2/networks/bublr-bikes',
 'id': 'bublr-bikes',
 'location': {'city': 'Milwaukee, WI',
  'country': 'US',
  'latitude': 43.0369,
  'longitude': -87.89667},
 'name': 'Bublr Bikes'}

In [17]:
companyURL = "http://api.citybik.es/v2/networks/bublr-bikes"

In [18]:
companyData = requests.get(companyURL).json()
companyData

{'network': {'company': ['BCycle, LLC'],
  'gbfs_href': 'https://gbfs.bcycle.com/bcycle_bublr/gbfs.json',
  'href': '/v2/networks/bublr-bikes',
  'id': 'bublr-bikes',
  'location': {'city': 'Milwaukee, WI',
   'country': 'US',
   'latitude': 43.0369,
   'longitude': -87.89667},
  'name': 'Bublr Bikes',
  'stations': [{'empty_slots': 12,
    'extra': {'address': '1420 N. Marshall St.',
     'ebikes': 0,
     'has_ebikes': True,
     'last_updated': 1712443082,
     'normal_bikes': 3,
     'rental_uris': {'android': 'https://www.bcycle.com/applink?system_id=bcycle_bublr&station_id=bcycle_bublr_3326&platform=android',
      'ios': 'https://www.bcycle.com/applink?system_id=bcycle_bublr&station_id=bcycle_bublr_3326&platform=iOS'},
     'renting': 1,
     'returning': 1,
     'uid': 'bcycle_bublr_3326'},
    'free_bikes': 3,
    'id': '4b562f25ad06507ac8ab4b8a118b0f24',
    'latitude': 43.0482,
    'longitude': -87.90086,
    'name': 'Ogden & Marshall',
    'timestamp': '2024-04-06T22:38:04.

In [21]:
len(companyData['network']['stations'])

117

In [22]:
companyData['network']['stations'][0]

{'empty_slots': 12,
 'extra': {'address': '1420 N. Marshall St.',
  'ebikes': 0,
  'has_ebikes': True,
  'last_updated': 1712443082,
  'normal_bikes': 3,
  'rental_uris': {'android': 'https://www.bcycle.com/applink?system_id=bcycle_bublr&station_id=bcycle_bublr_3326&platform=android',
   'ios': 'https://www.bcycle.com/applink?system_id=bcycle_bublr&station_id=bcycle_bublr_3326&platform=iOS'},
  'renting': 1,
  'returning': 1,
  'uid': 'bcycle_bublr_3326'},
 'free_bikes': 3,
 'id': '4b562f25ad06507ac8ab4b8a118b0f24',
 'latitude': 43.0482,
 'longitude': -87.90086,
 'name': 'Ogden & Marshall',
 'timestamp': '2024-04-06T22:38:04.102000Z'}

In [23]:
companyData['network']['stations'][1]

{'empty_slots': 9,
 'extra': {'address': '686 N. 8th St.',
  'ebikes': 1,
  'has_ebikes': True,
  'last_updated': 1712443082,
  'normal_bikes': 5,
  'rental_uris': {'android': 'https://www.bcycle.com/applink?system_id=bcycle_bublr&station_id=bcycle_bublr_3327&platform=android',
   'ios': 'https://www.bcycle.com/applink?system_id=bcycle_bublr&station_id=bcycle_bublr_3327&platform=iOS'},
  'renting': 1,
  'returning': 1,
  'uid': 'bcycle_bublr_3327'},
 'free_bikes': 6,
 'id': '05f41a87020da478a66df49c4d8ca954',
 'latitude': 43.03865,
 'longitude': -87.92193,
 'name': 'Central Library',
 'timestamp': '2024-04-06T22:38:04.103000Z'}